### Solving the Generalized Poisson Equation using Self consistent iterative procedure.

* In this notebook, we show how to solve the Generalized Poisson equation (GPe) using the self consistent iterative procedure.

* The GPe is defined as;
$$ \nabla \cdot \epsilon(r) \nabla \phi(r) = - 4 \pi \rho(r), $$

where $\epsilon(r)$ is the dielectric function, $\phi(r)$ is the electrostatic potential and $\rho(r)$ is the charge density.

#### The Charge Density, $\rho(r)$

The general formula for a three-dimensional Gaussian function centered at a position vector $\mathbf{p}$ with standard deviation $\sigma$ and total charge $Q$ over the space is given by:

$$
f(\mathbf{x}) = \frac{Q}{(2\pi \sigma^2)^{3/2}} \exp\left(-\frac{\|\mathbf{x} - \mathbf{p}\|^2}{2\sigma^2}\right)
$$

where:
* $\mathbf{x}$ is the position vector where the density is evaluated.
* $\mathbf{p}$ is the vector denoting the center of the Gaussian.
* $\sigma$ is the standard deviation of the Gaussian, reflecting its spread.
* $Q$ is the total charge, which scales the height of the Gaussian such that its integral over all space equals $Q$.
*  $|\mathbf{x} - \mathbf{p}|^2$ denotes the squared Euclidean distance between the point $\mathbf{x}$ and the center $\mathbf{p}$.


This formulation applies to the scenario where the Gaussian function is symmetric in all three dimensions around the center $\mathbf{p}$.

* Two equal but opposite charges defined in form of gaussians are placed at the center of the cell to form a dipole. 

#### The Dielectric Funtion, $\epsilon(r)$

* The dielectric function is defined in terms of the interface function. The interface function is the defined in form of an error function

#### Approach

* Transform the GPe to the form 

$$
\nabla^2 \phi(r) = -4 \pi \bigg[ \frac{\rho (r)}{\epsilon(r)} + \rho^{iter} (r) \bigg],
$$

where 
$$ \rho^{iter}. = \frac{1}{4 \pi} \nabla ln \epsilon(r) \cdot \nabla \phi(r)$$


* Since the $\rho^{iter}$ depends on $\phi(r)$, we employ Fast Fourier Transforms (FFTs) to calculate $\phi$ and use to find $\rho^{iter}$.

### Validating the results

The results were validated using the Onsager reaction field $E_{RF}$. This is defined as;

$$ E_{RF} = \frac{2 (\epsilon - 1) \bar{M}}{(2\epsilon - 1) r^3},$$

where $\bar{M}$ is the total dipole moment, $r$ is the radius and $\epsilon$ the dielectric function. 


### Dependencies

* To run this code, install $DFTpy$, a python module. This is an orbital free- Density Functional Theory code based on plane wave expansion of the electron density. 

#### Installation
The simplest way to install this would be through pip.

Do `pip install dftpy`


#### Importing necessary modules

In [16]:
import numpy as np
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.math_utils import ecut2nr
from scipy.special import erf
import matplotlib.pyplot as plt

In [17]:
def mean_squared_density(density: DirectField):
    """"
    Compute the mean squared sum of a scalar density
    """
    return np.sqrt(np.einsum("ijk->", (density*density))/density.grid.nnr)

In [18]:
def scalar_product(density1: DirectField, density2: DirectField):
    """
    Compute the scalar product of the density
    """
    return np.einsum("ijk->", (density1*density2))/density1.grid.nnr


### $\rho(r)$ as a gaussian

In [19]:
def gaussian_density(grid: DirectField, position: np.ndarray(shape=(3,)), sigma: float=0.5, charge: float=1.0):
    """
    Generate a scalar functions on a grid, the function is a Gaussian

    Input arguments:
        grid: DirectGrid with all the iformation on the cell
        position: center of the Gaussian
        sigma: spread of the Gaussian
        charge: total integral of the Gaussian

    Output:
        density: a DirectField scalar density on the grid
    """
    # Check the input values: sigma should be different from (greater than) zero
    if sigma <= 0. :
            raise ValueError("Gaussian spread should be positive")
    distance=np.einsum('ijkl->jkl',np.power(grid.r-np.broadcast_to(position,(grid.nr[2],grid.nr[1],grid.nr[0],3)).T,2))
    density=charge/(2*np.pi*sigma**2)**(3/2)*np.exp(-distance/2)

    return DirectField(grid=grid,data=density)

### The error function 

Here we construct the error function and its gradient to later be used to construct the interface function

In [20]:
def erf_density(grid: DirectField, position: np.ndarray(shape=(3,)), delta: float=0.5, width: float=1.0):
    """ home/denismulumba/trial/Environ/examples/programs/from_cube
    Generate a scalar functions on a grid, the function is a shifted error function (goes from 0 to 1)

    Input arguments:
        grid: DirectGrid with all the information on the cell
        position: center of the error function
        delta: spread of the transition from 0 to 1
        width: distance at which the function goes from 0 to 1

    Output:
        density: a DirectField scalar density on the grid
    """
    # width should be greater than zero, delta should be greater than zero
    if width <= 0. :
            raise ("Error function width should be positive")
    if delta <= 0. :
            raise ValueError("Error function spread should be positive")
    distance=np.sqrt(np.einsum('ijkl->jkl',np.power(grid.r - center.reshape(3,1,1,1),2)))
    density=(erf((distance-width)/delta)+1)*0.5
    return DirectField(grid=grid,data=density)

def erf_gradient(grid: DirectField, position: np.ndarray(shape=(3,)), delta: float=0.5, width: float=1.0):
    """
    Generate a vector function on a grid, the function is the gradient of a shifted error function

    Input arguments:
        grid: DirectGrid with all the information on the cell
        position: center of the error function
        delta: spread of the transition from 0 to 1
        width: distance at which the function goes from 0 to 1

    Output:
        gradient: a rank 1 (3 components) DirectField field on the grid
    """
    # width should be greater than zero, delta should be greater than zero
    if width <= 0. :
            raise ("Error function width should be positgrid.r - center.reshape(3,1,1,1)ive")
    if delta <= 0. :
            raise ValueError("Error function spread should be positive")
    distance = grid.r - center.reshape(3,1,1,1)
    abs_distance = np.sqrt(np.einsum('ijkl->jkl',np.power(distance,2)))
    abs_distance [ abs_distance < 1.e-10 ] = 1.e-10
    arg = (abs_distance-width)/delta
    return DirectField(grid=grid,rank=3,data=(1./np.sqrt(np.pi))*np.exp(-arg**2)*distance*1/(abs_distance*delta))

### Electrostatic field using FFTs

In [21]:
def calc_electrostatic_field(rho: DirectField):
    """
    Calculate the electrostatic field from a charge density
    Output: rho: DirectField with the charge density
    Output: v_h: DirectField with the electrostatic field
    """
    imag = 0 + 1j
    rho_g = rho.fft()
    grid_g = rho_g.grid
    grid_g.gg[0,0,0] = 1.# get the grid
    field_of_g = rho_g * 4 * np.pi * grid_g.g * imag / grid_g.gg
    return field_of_g.ifft(force_real=True)# calculate the residual

Generating the cell and grid. We will NOT be using the atoms or the electronic density from the quantum-mechanical simulation here

In [22]:
from ase.build import bulk
atoms = bulk('Al', 'sc', a=10.*0.52917745, cubic=True) # units in input are Angstrom, but the internal units are a.u.
ions = Ions.from_ase(atoms)
nr = ecut2nr(ecut=3000, lattice=ions.cell) # the cutoff is in Ry atomic units
grid = DirectGrid(lattice=ions.cell, nr=nr)
center=np.ones(3)*ions.cell.diagonal()*0.5 # assuming an orthorombic cell
icenter=nr//2

We generate a fictitious smooth density of charge at the center of the cell, using Gaussian functions.

In [23]:
# Properties of the gaussians
delta = 0.5
sigma = 0.5
# positive charge
pos_plus=center.copy()
pos_plus[0]+= delta
rho = gaussian_density(grid,pos_plus,sigma,charge=1.0)
# negative charge
pos_minus=center.copy()
pos_minus[0]-= delta
# we need to sum two charges, DirectField is not good for this, convert to nparray
rho += gaussian_density(grid,pos_minus,sigma,charge=-1.0)
dipole=(rho*grid.r).integral()

We generate the system-environment smooth interface interface functions, as a spherical region centered in the center of the cell

In [24]:
softness = 0.1
radius=3.0
interface=1-erf_density(grid,center,softness,radius)
interface_gradient=-erf_gradient(grid,center,softness,radius)
# plt.plot(interface[icenter[0],icenter[1],:]) # uncomment for debuggin and visualization
# plt.plot(interface_gradient[2,icenter[0],icenter[1],:]) # uncomment for debuggin and visualization
# print(interface.integral(),4/3*np.pi*radius**3) # check that the volume of the interface makes sense
# print(DirectField(grid=grid,data=np.sqrt(np.einsum('i...,i...->...',interface.gradient(),interface.gradient()))).integral(),4*np.pi*radius**2) # the surface also makes sense

From the interface function we can define the dielectric medium

In [25]:
epsilon0=80.
epsilon=(epsilon0-1.)*(1.-interface)+1.
epsilon_gradient=(1.0-epsilon0)*interface_gradient
grad_log_epsilon_analytical=epsilon_gradient/epsilon
grad_log_epsilon_numerical=np.log(epsilon).gradient()
# plt.plot(epsilon[icenter[0],icenter[1],:]) # uncomment for debuggin and visualization
# plt.plot(epsilon_gradient[2,icenter[0],icenter[1],:]) # uncomment for debuggin and visualization

Implement the fixed-point interative algorithm on the polarization charge

In [26]:
def fixed_point_solver( rho, epsilon, grad_log_epsilon, maxiter=100, tol=1.e-5, mixing=0.6 ):
    """
    Fixed point iterative solver on polarization charge. Requires the system charge density, the dielectric permittivity and
    the gradient of the logarithm of epsilon in input. Returns the polarization charge in output
    """
    # initialize the iterative polarization charge to zero
    pol_iter = DirectField(grid=rho.grid, data=np.zeros(rho.data.shape))
    # compute the fixed polarization charge from the part of the system charge leaking in the dielectric
    pol_fixed = (1.-epsilon)*rho/epsilon
    pol_fixed_charge = pol_fixed.integral() # total fixed polarization
    # compute the charge of the input density
    rho_charge = rho.integral()
    # start iterative solver
    iteration =0
    while iteration <  maxiter:
        # compute total charge density, including polarization charges
        rho_total = rho+pol_iter+pol_fixed
        # compute the electrostatic field from total charge
        field=calc_electrostatic_field(rho_total)
        # compute the new iterative polarization charge
        pol_new = np.einsum("lijk,lijk ->ijk", grad_log_epsilon, field / (4 * np.pi))
        pol_res=(mixing-1.)*(pol_iter-pol_new)
        pol_iter = pol_iter + pol_res
        pol_iter_charge = pol_iter.integral()
        # convergence is defined in terms of the quadratic mean of the residuals
        convergence=mean_squared_density(pol_res)
        iteration +=1
        print(iteration, convergence, rho_charge, pol_fixed_charge, pol_iter_charge)
        if convergence <= tol :#To stop the loop when the necessary condition is attained.
           return pol_fixed + pol_iter
    print('WARNING: convergence not achieved within',maxiter,' iterations')
    return pol_fixed + pol_iter

In [27]:
polarization = fixed_point_solver( rho, epsilon, grad_log_epsilon_numerical,tol=1e-5)

1 0.028614289856649888 -5.08027873109028e-06 5.016775248931301e-06 -1.1614775725042362e-08
2 0.014598167380765316 -5.08027873109028e-06 5.016775248931301e-06 -1.0987999802960061e-08
3 0.01107250340797813 -5.08027873109028e-06 5.016775248931301e-06 -8.448697640148159e-09
4 0.009303007463329474 -5.08027873109028e-06 5.016775248931301e-06 -7.275283060441341e-09
5 0.006129813954855474 -5.08027873109028e-06 5.016775248931301e-06 -7.300164153972007e-09
6 0.003765301004501747 -5.08027873109028e-06 5.016775248931301e-06 -7.68972320871446e-09
7 0.002838462467265056 -5.08027873109028e-06 5.016775248931301e-06 -7.974612143471957e-09
8 0.0020657658738114184 -5.08027873109028e-06 5.016775248931301e-06 -8.074534682604293e-09
9 0.0012164763718856499 -5.08027873109028e-06 5.016775248931301e-06 -8.067496688651922e-09
10 0.0007195077737756436 -5.08027873109028e-06 5.016775248931301e-06 -8.033156205643605e-09
11 0.0005678850168414508 -5.08027873109028e-06 5.016775248931301e-06 -8.008204135848426e-09
12 0

### Calculating the polarization field and comparing it against the Onsager model

In [28]:
polarization_field = calc_electrostatic_field(polarization)
print(polarization_field[:,icenter[0],icenter[1],icenter[2]])

[-2.89710573e-01 -1.52639989e-17 -1.30269221e-17]


In [29]:
field_onsager = - 2 * (epsilon0 - 1.)/(2*epsilon0 + 1)/radius**3 * dipole # calculate the electrostatic field from the Onsager solution
print(field_onsager)

[-2.90763737e-01  9.23262428e-07  9.23262428e-07]
